In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline


df = pd.read_csv("creditcard.csv")
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
#First we have to understand what the columns signify V1 to V7 signify the principal components of pca transformation. Amount is the transactions amount, time is the duration between each transaction. Class signifies fraud or not

In [6]:
data = df[df["Class"] == 1]
count = data.shape[0]
print(count)

492


In [7]:
#Only 492 rows of detected fraud, this implies a very unbalanced dataset, we need to address this before moving forward

In [8]:
data = df[df["Class"] == 1]
nan_count = data.isna().sum().sum()
print(nan_count)


0


In [9]:
#No NaN values, this allows us to use regression techniques


In [5]:
#To normalize the data we will use standard scaler , we will normalize time and amount as these can vary wildly, V1 to V28 should not be normalized as PCA already normalizes them
from sklearn.preprocessing import StandardScaler

#Copying the dataframe to avoid errors with the scaler
df1 = df.copy()


scaler = StandardScaler()


df1['scaled_amount'] = scaler.fit_transform(df1[['Amount']])


df1['scaled_time'] = scaler.fit_transform(df1[['Time']])

df1.drop(['Time', 'Amount'], axis=1, inplace=True)


In [6]:

X = df1.drop('Class', axis=1)
y = df1['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Handling class imbalance with smote
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_res, y_train_res)

rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_res, y_train_res)

y_pred_log_reg = log_reg.predict(X_test)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log_reg))
print("Logistic Regression AUC-ROC:", roc_auc_score(y_test, log_reg.predict_proba(X_test)[:, 1]))

y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print("Random Forest AUC-ROC:", roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:, 1]))

undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

rf_clf_under = RandomForestClassifier(random_state=42)
rf_clf_under.fit(X_train_under, y_train_under)

y_pred_rf_under = rf_clf_under.predict(X_test)
print("Undersampled Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf_under))
print("Undersampled Random Forest AUC-ROC:", roc_auc_score(y_test, rf_clf_under.predict_proba(X_test)[:, 1]))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.97      0.99     56962

Logistic Regression AUC-ROC: 0.9698482164390798
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.82      0.82        98

    accuracy                           1.00     56962
   macro avg       0.91      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962

Random Forest AUC-ROC: 0.9688299436966684
Undersampled Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1     